In [9]:
#!/usr/bin/env python
# coding: utf-8

from gurobipy import *
import numpy as np
import pandas as pd 
import numpy as np
os.chdir("C:\\Users\\uqrbean1\\OneDrive - The University of Queensland\\Documents\\ieee")

flist = [    
    "phase2_instance_small_0.txt",
    "phase2_instance_small_1.txt",
    "phase2_instance_small_2.txt",
    "phase2_instance_small_3.txt",
    "phase2_instance_small_4.txt",
    "phase2_instance_large_0.txt",
    "phase2_instance_large_1.txt",
    "phase2_instance_large_2.txt",
    "phase2_instance_large_3.txt",
    "phase2_instance_large_4.txt"    
        ]

for file in flist:
    count_data = np.genfromtxt(file,delimiter=' ',max_rows=1,usecols=(1,2,3,4,5),dtype=int)
    building_count = count_data[0]
    solar_count = count_data[1]
    battery_count = count_data[2]
    recurring_count = count_data[3]
    once_count = count_data[4]
    building_data = np.genfromtxt(file,delimiter=' ',skip_header=1,max_rows=building_count,usecols=(1,2,3),dtype=int)
    building_num = building_data[:,0]
    building_small = building_data[:,1]
    building_large = building_data[:,2]
    battery_data = np.genfromtxt(file,delimiter=' ',skip_header=1+solar_count+building_count,max_rows=battery_count,
                                 usecols=(1,3,4,5),dtype=float)

    with open(file, 'r') as f:
        file_lines = [line.rstrip() for line in f]

    df =  pd.DataFrame([string.split(' ') for string in file_lines])
    alldata = np.array(df)

    recur_data = alldata[alldata[:,0]=='r',]
    once_data = alldata[alldata[:,0]=='a',]

    once_data = np.array(df)

    recur_small = recur_data[recur_data[:,3]=='S']
    recur_large = recur_data[recur_data[:,3]=='L']

    T               = 532                                     # number of time periods
    timeperiods     = range(T)     
   
    nBuildings = building_count 
    buildings = range(nBuildings)
    
    smallActIdx = pd.to_numeric(recur_small[:,1])
    largeActIdx = pd.to_numeric(recur_large[:,1])

    smallAct = range(smallActIdx.shape[0])
    largeAct = range(largeActIdx.shape[0])

    ActEnergy = pd.to_numeric(recur_data[:,4])

    smallActPeriods = recur_small[:,5]
    largeActPeriods = recur_large[:,5]
    ActPeriods = pd.to_numeric(recur_data[:,5])

    ActPrec = pd.to_numeric(recur_data[:,6])

    ActRooms = pd.to_numeric(recur_data[:,2])

    Act = range(recurring_count)
       
    week1 = list(range(88,120)) + list(range(184,216)) + list(range(280,312)) + list(range(376,408)) + list(range(472,504))
    
    m = Model("monash_schedule")

    w            = m.addVars(Act, timeperiods, lb=-1, ub=1, name="w") # intermediate variable for differences
    v            = m.addVars(Act, timeperiods, name="v") # max v 
    s            = m.addVars(Act, timeperiods, vtype=GRB.BINARY, name="s") # binary allocation variable

    bu           = m.addVars(Act, buildings, lb=0, ub=3, vtype=GRB.INTEGER, name="roomlimit") # small_ac_buildings - sum to activity room count
    small_use       = m.addVars(timeperiods, buildings, lb=0, name="small_use")
    large_use       = m.addVars(timeperiods, buildings, lb=0, name="large_use")

    actLoad      = m.addVars(Act, timeperiods, name="load")
    load         = m.addVars(timeperiods, name="load1") # a variable load -- add net load and activity load and batteries 
    maxrecurload      = m.addVar(name="maxrecurload")
    
    recurload = m.addVars(timeperiods, name="recurload")
    
    z  = m.addVars(Act, lb=0,ub=T-1, name="z")
    z0 = m.addVars(Act, timeperiods, name="z0")
    
    for a in Act:
        m.addConstrs((z0[(a,t)] == t*v[(a,t)]) for t in timeperiods)

    for a in Act:
        m.addConstr(z[a] == max_(z0[a,t] for t in timeperiods))

    m.setObjective(maxrecurload, GRB.MINIMIZE)
        
    constr = m.addConstrs((w[(a,t)] == (s[(a,t)] - s[(a,t-1)]) for a in Act for t in range(1,T)), name="shift_starts_hat")
    constr = m.addConstrs((v[(a,t)] == max_(w[(a,t)], 0.0) for a in Act for t in range(1,T)), name="shift_starts")
    constr = m.addConstrs((v[(a,0)] == s[(a,0)] for a in Act), name="shift_starts_init")
    constr = m.addConstrs((quicksum([v[(a,t)] for t in timeperiods]) <= 1 for a in Act), name="shift_start_max")

    constr = m.addConstrs((recurload[t] == (quicksum([actLoad[(a,t)] for a in Act])) for t in week1), name="netloada")
    constr = m.addConstr(maxrecurload == max_([recurload[t] for t in week1]),name="get_max_load") ## for the objective function
    
    constr = m.addConstrs((quicksum([s[(a,t)] for t in timeperiods]) == ActPeriods[a] for a in Act), name="col")

    constr = m.addConstrs((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t)] for a in smallActIdx]) for b in buildings for t in timeperiods ),name="bus")
    constr = m.addConstrs((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t)] for a in largeActIdx]) for b in buildings for t in timeperiods ),name="bul")

    constr = m.addConstrs((small_use[(t,b)] <= building_small[b] for t in timeperiods for b in buildings), name="small_lim")
    constr = m.addConstrs((large_use[(t,b)] <= building_large[b] for t in timeperiods for b in buildings), name="large_lim")

    const = m.addConstrs((quicksum([bu[(a,b)] for b in buildings]) == ActRooms[a] for a in smallActIdx ),name="smallrooms")
    const = m.addConstrs((quicksum([bu[(a,b)] for b in buildings]) == ActRooms[a] for a in largeActIdx ),name="largerooms")

    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(0,88)]) == 0 for a in Act),name="zero1")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(120,184)]) == 0 for a in Act),name="zero2")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(216,280)]) == 0 for a in Act),name="zero3")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(312,376)]) == 0 for a in Act),name="zero4")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(408,472)]) == 0 for a in Act),name="zero5")
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(504,532)]) == 0 for a in Act),name="zero6")
    for a in Act:
        ro = pd.to_numeric(recur_data[a,6:])
        co = int(ro[0])
        for i in range(co):
            m.addConstr((z[a] - z[int(ro[i+1])] >= 65),name="prec")

    constr = m.addConstrs((actLoad[(a,t)] == ActEnergy[a]*ActRooms[a]*s[(a,t)]) for a in Act for t in timeperiods)

    #################################     
        
    sfile = file.replace("phase2","p2sol\\flat_") # the given start point
    sfile2 = sfile.replace("_instance_","") # the given start point
    sfile3 = sfile2.replace("_","") # the given start point
    sfile4 = sfile3.replace("txt","mps") # the given start point
    m.write(sfile4)

In [6]:
os.getcwd()

'C:\\Users\\uqrbean1\\OneDrive - The University of Queensland\\Documents\\ieee'